<p style="font-size: 30px; font-weight: bold;">Lock Based Data Structures</p>

# General Design Guidelines - Concurrency

* Multiple threads can access the data structure concurrently
    * May perform the same or distinct operations
* Each thread will see a consistent view of the data structure
    * No data lost or corrupted
    * All invariants upheld
    * No problematic race condition
* In general, a data structure will be safe only for certain types of concurrent access

<img src="img/design_concurrency.png" alt="Design for concurrency" width="80%" style="margin: 0 auto;">

## Guidelines

* No thread shall see invariants broken by another thread
* Avoid race conditions inherent in the interface by providing functions for complete operations rather than for operation steps
* Pay attention to how the data structure behaves in the presence of exceptions to ensure that the invariants are not broken
* Minimize the opportunities for deadlock by restricting the scope of locks and avoiding nested locks where possible

## Questions to ponder

* If one thread is accessing the data structure through a particular function – which functions are safe to call from other threads?
    * Constructors and destructors generally require exclusive access
    * How about assignment, swap(), or copy constructors?
* Can the scope of locks be restricted to parts of a function?
* Can different parts of the data structure be protected with different mutexes?
* Do all operations require the same level of protection?
* Can a simple change to the data structure improve opportunities for concurrency without affecting the operational semantics?

# Stack

## Ideas for design

1. Pass in a reference to retrieve the popped item
    * Requires calling code to construct an instance of the stack’s value type
    * Sometimes expensive or impossible (e.g., lack of constructor arguments)
2. Return the popped item as value
    * Requires a no-throw copy constructor or move constructor to avoid exception safety problem
    * Limiting because not provided by all possible value types
3. Return a pointer to the popped item
    * Requires managing the memory allocated to the object
    * Consider using std::shared_ptr to avoid memory leaks
4. Provide both option 1 and either option 2 or 3

```cpp
template<typename T>
class ts_stack {
private:
    std::stack<T> data;
    mutable std::mutex m;
public:
    
    ts_stack(){}
    
    ts_stack(const ts_stack& other) {
        std::lock_guard<std::mutex> lock(other.m);
        data = other.data;
    }
    ts_stack& operator=(const ts_stack&) = delete;
    
    void push(T new_value) {
        std::lock_guard<std::mutex> lock(m);
        data.push(std::move(new_value));
    }
    
    bool empty() const {
        std::lock_guard<std::mutex> lock(m);
        return data.empty();
    }
    
    std::shared_ptr<T> pop() {
        std::lock_guard<std::mutex> lock(m);
        if(data.empty()) throw empty_stack();
        std::shared_ptr<T> const res(
        std::make_shared<T>(std::move(data.top())));
        data.pop();
        return res;
    }
    void pop(T& value) {
        std::lock_guard<std::mutex> lock(m);
        if(data.empty()) throw empty_stack();
        value = std::move(data.top());
        data.pop();
    }
}
```

## Review of interface

* Basic thread safety by protecting each member function call with lock
    * Exception: constructor and destructor
    * Not big issue because those function are called only once
* Potential for race condition between `empty()` and either of the `pop()` versions – not problematic because `pop()` explicitly checks whether stack empty

## Potential sources of exceptions

* Locking a mutex – safe because no data has been modified
* `push()` : `data.push()` – safe because `std::stack<>` guarantees it
* `pop()`
    * `empty_stack()` exception – safe because no data has been modified
    * `data.pop()` – safe because std::stack<> guarantees it
    * First overload of pop() – Creation of res safe because no data has been modified
    * First & second overload of pop() – Copy or move assignment op. safe because no data has been modified
* `empty()` : does not modify any data

## Potential sources of deadlock

Calling user code while holding a lock
* First `pop()` – copy or move constructor of value type
* Second `pop()` – copy assignment or move assignment of value type
* Sensible to require that users of stack will take care of it

## Performance

* Only one thread can do work on the stack at a time
    * Effectively serializes operations on the stack
* No provision for threads that need to wait for stack to be filled
    * Repeated calls to empty() – busy waiting
    * Alternative – external wait and notify code

# Queue with `std::queue`

<img src="img/queue_thread.png" alt="Queue Thread" width="60%">

## A thread-safe queue with condition variables

* Combine `front()` and `pop()` into a single function call
* Two variants of `pop()`
    * `try_pop()` – always returns immediately with an indication of success
    * `wait_and_pop()` – waits until there is a value to retrieve

## Thread-safe Queue

```cpp
class ts_queue
{
private:
    mutable std::mutex mut;
    std::queue<T> data_queue;
    std::condition_variable data_cond;
public:
    ts_queue() {}
    ts_queue(ts_queue const& other) {
        std::lock_guard<std::mutex> lk(other.mut);
        data_queue = other.data_queue;
    }
    
    void push(T new_value) {
        std::lock_guard<std::mutex> lk(mut);
        data_queue.push(new_value);
        data_cond.notify_one();
    }
    
    bool empty() const {
        std::lock_guard<std::mutex> lk(mut);
        return data_queue.empty();
    }
    
    void wait_and_pop(T& value) {
        std::unique_lock<std::mutex> lk(mut);
        data_cond.wait(lk, [this]{return !data_queue.empty();});
        value=data_queue.front();
        data_queue.pop();
    }
    
    std::shared_ptr<T> wait_and_pop() {
        std::unique_lock<std::mutex> lk(mut);
        data_cond.wait(lk, [this]{return !data_queue.empty();});
        std::shared_ptr<T> res(std::make_shared<T>(data_queue.front()));
        data_queue.pop();
        return res;
    }
    
    bool try_pop(T& value) {
        std::lock_guard<std::mutex> lk(mut);
        if(data_queue.empty)
            return false;
        value = data_queue.front();
        data_queue.pop();
        return true;
    }
    
    std::shared_ptr<T> try_pop() {
        std::lock_guard<std::mutex> lk(mut);
        if(data_queue.empty())
            return std::shared_ptr<T>();
        std::shared_ptr<T> res(std::make_shared<T>(data_queue.front()));
        data_queue.pop();
        return res;
    }
};
```

## Exception Safety

* If a thread throws exception in `wait_and_pop()`, e.g., when a shared pointer is constructed, none of the other threads will be woken
* Three possible solutions
    * Use `notify_all` – at the cost of most of them going back to sleep
    * Let `wait_and_pop()` call `notify_one()` if an exception is thrown
    * Move shared-pointer initialization to push call – requires storing shared pointers instead of plain values in the queue

## Review

* One protected data item - data_queue
* Essentially thread-safe wrapper around std::queue
    * Requires only one mutex
* Thread safe but concurrency limited
* Alternative: fine-grained locking
    * Requires control of implementation details

# Simple Queue based on Single-Linked List

<img src="img/simple_queue_thread.png" alt="Simple Queue Thread" width="60%" style="margin: 0 auto;">

* Items are added at the tail
* Items are removed from the head
* When the list is empty – both the head and the tail pointers are `NULL`

```cpp
template<typename T>
class queue {
public:
    queue(): tail(nullptr) {}
    queue(const queue& other) = delete;
    queue& operator=(const queue& other) = delete;
    std::shared_ptr<T> try_pop();
    void push(T new_value);
private:
    struct node {
        T data;
        std::unique_ptr<node> next;
        node(T data_) :
        data(std::move(data_)) {}
    };
    std::unique_ptr<node> head;
    node* tail;
};
```

```cpp
// try_pop method for sequential execution
std::shared_ptr<T> queue::try_pop() {
    if(!head) {
        return std::shared_ptr<T>();
    }
    std::shared_ptr<T> const res(std::make_shared<T>(std::move(head->data)));
    std::unique_ptr<node> const old_head = std::move(head);
    head = std::move(old_head->next);
    if(!head) tail = nullptr;
    return res;
}
```

```cpp
void queue::push(T new_value) {
    std::unique_ptr<node> p(new node(std::move(new_value)));
    node* const new_tail = p.get();
    if(tail) {
        tail->next = std::move(p);
    }
    else {
        head = std::move(p);
    }
    tail = new_tail;
}
```

## Parallelization

Idea – use separate mutexes for head and tail
* `push()` can modify head and tail
    * Requires locking both mutexes
    * Unfortunate, but not a big problem
* Both `push()` and `try_pop()` access next pointer of a node
    * `push()` updates `tail->next`
    * `try_pop()` reads `head->next`
    * If there is a single item in the queue, then `head == tail`
    * Finding out requires locking the same mutex in `push()` and `try_pop()`

<img src="img/simple_queue_thread_dummy.png" alt="Simple Queue Thread" width="60%" style="margin: 0 auto;">

```cpp
// try_pop method for parallel execution not yet parallel consistency
std::shared_ptr<T> queue::try_pop() {
    if(head.get() == tail) {
        return std::shared_ptr<T>();
    }
    std::shared_ptr<T> const res(std::move(head->data));
    std::unique_ptr<node> const old_head = std::move(head);
    head = std::move(old_head->next);
    return res;
}
```

```cpp
// push method for parallel execution
void queue::push(T new_value) {
    std::shared_ptr<T> new_data( std::make_shared<T>(std::move(new_value)) );
    std::unique_ptr<node> p(new node);
    tail->data = new_data;
    node* const new_tail = p.get();
    tail->next = std::move(p);
    tail = new_tail;
}
```

## Discussion

* `push()` now accesses only tail not head
* `try_pop()` accesses both head and tail – but tail is needed only for the initial comparison
    * Lock will be needed only for a short time
* Main advantage – `try_pop()` and `push()` will never operate on the same node
    * No need for overarching mutex
    * Creates significantly better opportunities for concurrency

## a thread-safe queue with fine-grained locking

```cpp
template<typename T>
class ts_queue {
private:
    std::mutex head_mutex;
    std::mutex tail_mutex;
    node* get_tail();
    std::unique_ptr<node> pop_head();
    
    struct node {
        std::shared_ptr<T> data;
        std::unique_ptr<node> next;
        node(T data_) :
        data(std::make_shared<T>(std::move(data_))) {}
    };
    
    std::unique_ptr<node> head;
    node* tail;
public:
    ts_queue():
    head(new node),tail(head.get()) {}
    ts_queue(const ts_queue& other) = delete;
    ts_queue& operator=(const ts_queue& other) = delete;
    std::shared_ptr<T> try_pop();
    void push(T new_value);
};
```

```cpp
node* get_tail() {
    std::lock_guard<std::mutex> tail_lock(tail_mutex);
    return tail;
}
```

```cpp
std::unique_ptr<node> pop_head() {
    std::lock_guard<std::mutex> head_lock(head_mutex);
    if(head.get() == get_tail()) {
        return nullptr;
    }
    std::unique_ptr<node> const old_head = std::move(head);
    head = std::move(old_head->next);
    return old_head;
}
```

```cpp
std::shared_ptr<T> try_pop() {
    std::unique_ptr<node> old_head = pop_head();
    return old_head ? old_head->data : std::shared_ptr<T>();
}
```

```cpp
void push(T new_value) {
    std::shared_ptr<T> new_data(std::make_shared<T>(std::move(new_value)));
    std::unique_ptr<node> p(new node);
    node* const new_tail = p.get();
    std::lock_guard<std::mutex> tail_lock(tail_mutex);
    tail->data = new_data;
    tail->next = std::move(p);
    tail = new_tail;
}
```

## Invariants

* `tail->next == nullptr` 
* `tail->data == nullptr`
* `head == tail` implies an empty queue
* A single-element queue has `head->next == tail`
* For each node `x` in the queue, where `x != tail`, `x->data` points to an instance of `T` and `x->next` points to the next node in the queue
* `x->next == tail` implies `x` is the last item node the queue
* Following the next nodes from head will eventually yield tail

### Race conditions / data races – `push()`

* All modifications to the data structure protected by `tail_mutex`
* Modifications uphold invariants
    * New tail node is empty node
    * data and next correctly set for old tail node (last real node in the queue)

### Race conditions / data races – `try_pop()`

* Locking `tail_mutex` avoids data race when reading tail
    * The mutex ensures that try_pop() either sees the old or a new value of tail and the new data attached to the previous value of tail
    * Note that push can only add data but not remove it
* Also important that `get_tail()` is called inside the lock on head mutex

## Concurrency

* More is done outside locks in comparison to wrapper implementation
* Expensive memory allocations in push() occur outside locks
    * Adding node to the queue involves only simple pointer assignments
    * Improves concurrency between different calls to push()
* `try_pop()` holds tail_mutex only for short time
    * Improves concurrency between try_pop() and push()
* Operations under protection of head_mutex are cheap
    * Expensive delete operations in try_pop() occur outside lock
    * Improves concurrency between different calls to try_pop()